# FLAML (Fast and Lightweight AutoML) demos

In [ ]:
!pip3 install flaml

     |████████████████████████████████| 130 kB 30.9 MB/s 
     |████████████████████████████████| 2.0 MB 38.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


## Classifier

In [ ]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                     header=None, names=('sepal length', 'sepal width', 'petal length', 'petal width', 'species'))
df

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df.species = le.fit_transform(df.species)
labels = le.classes_

labels

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

x_train = train.drop(['species'], axis=1)
y_train = train.species
x_test = test.drop(['species'], axis=1)
y_test = test.species

In [ ]:
from flaml import AutoML

clf = AutoML()
clf.fit(x_train, y_train, task='classification', time_budget=10)

[flaml.automl: 12-02 14:02:08] {1926} INFO - task = classification
[flaml.automl: 12-02 14:02:08] {1928} INFO - Data split method: stratified
[flaml.automl: 12-02 14:02:08] {1932} INFO - Evaluation method: cv
[flaml.automl: 12-02 14:02:08] {1999} INFO - Minimizing error metric: log_loss
[flaml.automl: 12-02 14:02:08] {2051} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 12-02 14:02:08] {2291} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-02 14:02:08] {2405} INFO - Estimated sufficient time budget=586s. Estimated necessary time budget=13s.
[flaml.automl: 12-02 14:02:08] {2485} INFO -  at 0.1s,	estimator lgbm's best error=0.6695,	best estimator lgbm's best error=0.6695
[flaml.automl: 12-02 14:02:08] {2291} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-02 14:02:08] {2485} INFO -  at 0.2s,	estimator lgbm's best error=0.6695,	best estimator lgbm's best error=0.6695
[flaml.automl: 12-02 14:0

In [ ]:
print('Best ML leaner:', clf.best_estimator)
print('Best hyperparmeter config:', clf.best_config)

Best ML leaner: extra_tree
Best hyperparmeter config: {'n_estimators': 4, 'max_features': 0.9692029582222275, 'max_leaves': 6, 'criterion': 'entropy'}


In [ ]:
predicted = clf.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted, target_names=labels))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30



## Regressor

In [ ]:
import pandas as pd

df = pd.read_csv('https://github.com/PacktPublishing/Automated-Machine-Learning-with-AutoKeras/raw/main/boston.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from flaml import AutoML

reg = AutoML()
reg.fit(dataframe=train, label='MEDV', task='regression', estimator_list=['lgbm'], time_budget=60)

[flaml.automl: 12-02 14:02:43] {1926} INFO - task = regression
[flaml.automl: 12-02 14:02:43] {1928} INFO - Data split method: uniform
[flaml.automl: 12-02 14:02:43] {1932} INFO - Evaluation method: cv
[flaml.automl: 12-02 14:02:43] {1999} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-02 14:02:43] {2051} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 12-02 14:02:43] {2291} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-02 14:02:43] {2405} INFO - Estimated sufficient time budget=585s. Estimated necessary time budget=1s.
[flaml.automl: 12-02 14:02:43] {2485} INFO -  at 0.1s,	estimator lgbm's best error=0.6264,	best estimator lgbm's best error=0.6264
[flaml.automl: 12-02 14:02:43] {2291} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-02 14:02:43] {2485} INFO -  at 0.2s,	estimator lgbm's best error=0.6264,	best estimator lgbm's best error=0.6264
[flaml.automl: 12-02 14:02:43] {2291} INFO - iteration 2, current learner lgbm
[flaml.automl: 

In [ ]:
print('Best ML leaner:', reg.best_estimator)
print('Best hyperparmeter config:', reg.best_config)

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 403, 'num_leaves': 11, 'min_child_samples': 7, 'learning_rate': 0.038776255223058866, 'log_max_bin': 10, 'colsample_bytree': 0.7717990033856998, 'reg_alpha': 0.000996743598444551, 'reg_lambda': 0.1313580710283504}


In [ ]:
x_test = test.drop('MEDV', axis=1)
y_test = test.MEDV

predicted = reg.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

print('Prection R2:', r2_score(y_test, predicted).round(3))
print('Prection MAE:', mean_absolute_error(y_test, predicted).round(3))

Prection R2: 0.906
Prection MAE: 1.803
